In [673]:
import os
import json
import pandas as pd
from dotenv import load_dotenv
import requests
import logging
import sys
from IPython.display import HTML
import warnings
warnings.filterwarnings("ignore")

In [674]:
''' pip install python-dotenv'''
# load_dotenv() # will search for .env file in local folder and load variables
# Reload the variables from your .env file, overriding existing ones
load_dotenv(".env", override=True)

True

In [675]:
# 로깅 설정
logging.basicConfig(
    level=logging.INFO,
    # format='%(asctime)s - %(name)s - %(levelname)s - %(message)s',
    format='[%(asctime)s] [%(levelname)s] [%(module)s] [%(funcName)s] %(message)s',
    handlers=[
        logging.StreamHandler(sys.stdout)
    ]
)

In [676]:
logger = logging.getLogger("zinc-search-call")
# Display the log message with a brown background

In [677]:
''' https://restfulapi.net/http-status-codes/ '''
http_status_code = {
    200 : 'Indicates that the request has succeeded.',
    201 : 'Indicates that the request has succeeded and a new resource has been created as a result.',
    400 : 'The server could not understand the request due to incorrect syntax. The client should NOT repeat the request without modifications.',
    401 : 'Unauthorized rquests. Insufficient permissions',
    404: 'the server could not find the requested resource (web page)',
    405 : 'the server has received a request using a valid HTTP method, but the target resource does not support that specific method.',
    422 : 'server understood the content type of the request body and the syntax of the request body was correct, but it was unable to process the contained instructions.',
    403 : 'Unauthorized request. Insufficient permissions. The client does not have access rights to the content. ',
    500 : 'The server encountered an unexpected condition that prevented it from fulfilling the request.'
}

In [678]:
# Define a styling function
def highlight_failed_rows(row):
    # Apply red background if 'Status' is 'Fail'
    if row['Age'] < 30:
        return ['background-color: red; font-weight: bold'] * len(row)
    else:
        return [''] * len(row)

In [685]:
''' sample '''
''' https://docs.kanaries.net/ko/topics/Pandas/pandas-add-column '''
data = {
    'Name': ['Alice', 'Bob', 'Charlie', 'David'],
    'Age': [25, 30, 35, 40]
}
 
df = pd.DataFrame(data)
# Apply the styling function to the DataFrame
df = df.style.apply(highlight_failed_rows, axis=1)
# df.head(10)
display(df)

,Name,Age
0,Alice,25
1,Bob,30
2,Charlie,35
3,David,40


In [680]:
def bulk_zinc_search(url):
    ''' bulk for the ZINC Search Engine '''
    url = '{}'.format(url)
    headers = {
           'Content-type': 'application/json',
           'Authorization' : 'Basic {}'.format(os.getenv("BASIC_AUTH_ZINC_SEARCH")),
           'Connection': 'close'
    }

    # Index name
    INDEX_NAME = "my_index"

    # Documents to be ingested
    documents = [
        {"id": "1", "title": "First Document", "content": "This is the content of the first document."},
        {"id": "2", "title": "Second Document", "content": "Another document for bulk ingestion."},
        {"id": "3", "title": "Third Document", "content": "The final document in this example."},
    ]

    # Create the bulk payload in NDJSON format
    bulk_payload_lines = []
    for doc in documents:
        # Action line for indexing
        action_line = {"index": {"_index": INDEX_NAME, "_id": doc["id"]}}
        bulk_payload_lines.append(json.dumps(action_line))
        # Source document line
        bulk_payload_lines.append(json.dumps(doc))

    bulk_payload = "\n".join(bulk_payload_lines) + "\n"  # Add a final newline
 
    # Send the bulk request
    try:
        response = requests.post(f"{url}", headers=headers, data=bulk_payload)
        response.raise_for_status()  # Raise an exception for bad status codes (4xx or 5xx)
    
        print("Bulk ingestion successful:")
        print(response.status_code)
        # print(json.dumps(response.json(), indent=2))

        # Example: Indexing a document
        doc_data = {"title": "My first document", "content": "This is some content."}
        '''
        response = requests.post(f"http://{os.getenv('api_end_point')}:4080/api/{INDEX_NAME}/_doc/", 
                                 headers=headers, 
                                 data=json.dumps(doc_data))
        '''
        response = requests.put(f"http://{os.getenv('api_end_point')}:4080/api/{INDEX_NAME}/_doc/1", 
                                 headers=headers, 
                                 data=json.dumps(doc_data))
        print(response.json())
    
    except requests.exceptions.RequestException as e:
        print(f"Error during bulk ingestion: {e}")
        if hasattr(e, 'response') and e.response is not None:
            print(f"Response content: {e.response.text}")

In [681]:
bulk_zinc_search("http://{}:4080/es/_bulk".format(os.getenv("api_end_point")))

Bulk ingestion successful:
200
{'message': 'ok', 'id': '1', '_id': '1', '_index': 'my_index', '_version': 1, '_seq_no': 0, '_primary_term': 0, 'result': 'created'}


In [682]:
def get_zinc_search(url):
    ''' Search results for the ZINC Search Engine '''
    url = '{}'.format(url)
    headers = {
           'Content-type': 'application/json',
           'Authorization' : 'Basic {}'.format(os.getenv("BASIC_AUTH_ZINC_SEARCH")),
           'Connection': 'close'
    }

    # payload = {
    #     "query" : {
    #         "match_all" : {}
    #     }
    # }
    payload = {
        # "search_type": "match",
        # "query": {
        #     "term": "Paris",  # The term to search for
        #     "field": "_all",  # The field to search in (or a specific field name)
        #     # "start_time": "2021-06-02T14:28:31.894Z", # Optional: time range filtering
        #     # "end_time": "2021-12-02T15:28:31.894Z"
        # },
        "search_type": "querystring",
        "query": {
            "term": "first"
        },
        "from": 0,  # Starting point for pagination
        "max_results": 10,  # Number of results to return
        "_source": [],  # List of fields to return (empty for all fields),
        "highlight": {
            "fields": {
              "*": {}
            }
        }
    }

    response = requests.post(url, json=payload, headers=headers, verify=False)
    logging.info(f"{response.status_code} - {http_status_code.get(response.status_code, '')}")
    logging.info(f"{json.dumps(response.json(), indent=2)}")

In [683]:
get_zinc_search("http://{}:4080/api/my_index/_search".format(os.getenv("api_end_point")))

2025-12-11 17:52:17,688 - root - INFO - 200 - Indicates that the request has succeeded.
2025-12-11 17:52:17,689 - root - INFO - {
  "took": 0,
  "timed_out": false,
  "_shards": {
    "total": 3,
    "successful": 3,
    "skipped": 0,
    "failed": 0
  },
  "hits": {
    "total": {
      "value": 1
    },
    "max_score": 0.5767534391980533,
    "hits": [
      {
        "_index": "my_index",
        "_type": "_doc",
        "_id": "1",
        "_score": 0.5767534391980533,
        "@timestamp": "2025-12-11T22:50:14.033972736Z",
        "_source": {
          "@timestamp": "2025-12-11T22:50:14.033972736Z",
          "content": "This is some content.",
          "title": "My first document"
        }
      }
    ]
  }
}


In [684]:
HTML('<pre style="background-color: #A52A2A; color: white;">Job is being performed correctly</pre>')